<a href="https://colab.research.google.com/github/Rudr16a/Attendance-/blob/main/Attendance_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install basic dependencies
!pip install pandas streamlit opencv-contrib-python numpy face_recognition cmake dlib

# Streamlit-specific dependencies
!pip install pyngrok --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.6 MB/s eta 0:00:00
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=1a485281feec6aa58270b985e499958722e44996ddb318877935daa7d9a13305
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstall

In [3]:
!pip install streamlit

In [4]:
import pandas as pd
import streamlit as st
import cv2
import face_recognition
import numpy as np
import csv
from datetime import datetime
import os
import time
import base64

# Set the Streamlit page configuration
st.set_page_config(page_title="Attendance System", page_icon=":guardsman:", layout="wide")

# Create a title and a subheader
st.title("Welcome to Attendance System")
st.subheader("Powered by Face Recognition")

# Load known images and create encodings
known_face_encodings = []
known_faces_names = []

# Make sure the "faces" directory exists and contains the image files
if not os.path.exists("faces"):
    st.error('"faces" directory not found! Please upload the images first.')
else:
    for file in os.listdir("faces"):
        if file.endswith(".jpg"):
            name = file.split(".")[0]
            image = face_recognition.load_image_file(os.path.join("faces", file))
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            known_faces_names.append(name)

# Initialize list to store students
students = known_faces_names.copy()
current_date = datetime.now().strftime("%Y-%m-%d")

# Adjust tolerance level for face recognition
tolerance = 0.7

# Create a button to start the attendance process
if st.button("Start Attendance"):
    # Open a video capture
    videocap = cv2.VideoCapture(0)

    # Create a CSV file to store attendance records
    with open(current_date + '.csv', 'w+', newline='') as f:
        lnwriter = csv.writer(f)
        st.info("Starting attendance... Press 'Stop Attendance' to finish.")

        while True:
            ret, frame = videocap.read()
            if not ret:
                st.error("No video feed detected! Please ensure your camera is working.")
                break

            # Find face locations and encodings in the frame
            face_locations = face_recognition.face_locations(frame)
            face_encodings = face_recognition.face_encodings(frame, face_locations)
            frame_height, frame_width = frame.shape[:2]

            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance)
                name = "Unknown"
                if True in matches:
                    best_match_index = np.argmax(matches)
                    name = known_faces_names[best_match_index]
                if name in students:
                    students.remove(name)
                    current_time = datetime.now().strftime("%H:%M:%S")
                    lnwriter.writerow([name, current_time])
                    f.flush()

                    # Resize and display the image in Streamlit
                    frame = cv2.resize(frame, (int(frame_width / 2), int(frame_height / 2)))
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    st.image(frame, use_column_width=True)
                    st.write(f"{name} has been marked as present.")
                    time.sleep(2)

            # Check if the "Stop Attendance" button is pressed
            if st.button("Stop Attendance"):
                break

        videocap.release()
        st.success("Attendance process complete.")
        st.info(f"Attendance records have been saved in {current_date}.csv")

        # Download Attendance
        with open(current_date + '.csv', 'r') as f:
            csv_data = f.read()
            b64 = base64.b64encode(csv_data.encode()).decode()
            href = f'<a href="data:file/csv;base64,{b64}" download="{current_date}.csv">Download CSV File</a>'
            st.markdown(href, unsafe_allow_html=True)


2024-08-09 12:30:49.908 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
